In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

from scipy.stats import spearmanr

from uncertain_x.line_fit import uncertain_x

In [2]:
%matplotlib notebook

# Volume Limited Sample of Normal SNe Ia

This notebook provides summary statistics when examining a volume limited sample of "normal" SNe Ia from the ZTF 2018 early sample.

In [3]:
info_path="../../forced_lightcurves/sample_lc_v2/"
salt_df = pd.read_csv(info_path + "../../Nobs_cut_salt2_spec_subtype_pec.csv")
res = pd.read_csv('results_40percent.csv')
res_tsquared = pd.read_csv('results_40_tsquared.csv')

In [4]:
normal = np.where((salt_df['Ia subtype'] != 'Ia-CSM') & 
                  (salt_df['Ia subtype'] != 'SC') & 
                  (salt_df['Ia subtype'] != 'SC*') & 
                  (salt_df['Ia subtype'] != '86G-like') & 
                  (salt_df['Ia subtype'] != '02cx-like'))
good = np.where(res.final_selection.values == 1)
has_host_z = np.where(salt_df.z_source != 'Msl')

normal_good = np.intersect1d(normal, good)
normal_good_host = np.intersect1d(normal_good, has_host_z)

# rearange for plotting
normal_plot = np.setdiff1d(normal[0], normal_good)
normal_good_plot = np.setdiff1d(normal_good, normal_good_host)

In [5]:
low_z = np.where(salt_df.z_adopt < 0.06)


normal_lowz = np.intersect1d(low_z, normal)
print(len(normal_lowz))
normal_good_lowz = np.intersect1d(low_z, normal_good)
print(len(normal_good_lowz))
normal_good_host_lowz = np.intersect1d(low_z, normal_good_host)
print(len(normal_good_host_lowz))

28
16
15


In [6]:
t_rise_16 = res.t_rise_16.values.copy()
t_rise_50 = res.t_rise_50.values.copy()
t_rise_84 = res.t_rise_84.values.copy()

sigma_i = (t_rise_84-t_rise_16)/2
w_i = (sigma_i)**(-2)

In [7]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_lowz], t_rise_50[normal_lowz], 
                            salt_df.x1_salt2_unc.values[normal_lowz], sigma_i[normal_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.0, intercept_guess=17)
print('the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_lowz])
print('the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))
salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_lowz], t_rise_50[normal_good_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_lowz], sigma_i[normal_good_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=0.85, intercept_guess=17)
print('    the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('    the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_lowz])
print('    the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_host_lowz], t_rise_50[normal_good_host_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_host_lowz], sigma_i[normal_good_host_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=0.85, intercept_guess=17)
print('\t  the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('\t  the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_host_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_host_lowz])
print('\t  the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))

The following are summary statistics for rise time:
mean, median, weighted mean: 17.27950, 17.36291, 18.60908
std, sMAD, sigma w_mean: 2.26067, 2.45129, 0.05104
the slope is: 1.3532 +/- 0.0465
the intercept is: 18.0829 +/- 0.5273
the residual scatter is: 1.6949 d
 
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 18.33064, 18.03953, 18.90659
    std, sMAD, sigma w_mean: 1.75491, 2.19984, 0.05318
    the slope is: 0.9693 +/- 0.0041
    the intercept is: 18.3813 +/- 0.0546
    the residual scatter is: 1.4232 d
 
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 18.47577, 18.34232, 18.94635
	  std, sMAD, sigma w_mean: 1.71424, 2.17898, 0.05356
	  the slope is: 0.9277 +/- 0.0032
	  the intercept is: 18.4191 +/- 0.0519
	  the residual scatter is: 1.4050 d


In [8]:
flat_alpha_g_16 = res.flat_alpha_g_16.values.copy()
flat_alpha_g_50 = res.flat_alpha_g_50.values.copy()
flat_alpha_g_84 = res.flat_alpha_g_84.values.copy()

sigma_i = (flat_alpha_g_84-flat_alpha_g_16)/2
w_i = (sigma_i)**(-2)

In [9]:
print('The following are summary statistics for alpha_g:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_lowz]), np.median(flat_alpha_g_50[normal_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_g_50[normal_lowz] - np.median(flat_alpha_g_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_lowz]), np.median(flat_alpha_g_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_lowz] - np.median(flat_alpha_g_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_host_lowz]), np.median(flat_alpha_g_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_host_lowz] - np.median(flat_alpha_g_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_g:
mean, median, weighted mean: 2.10809, 2.21567, 2.05053
std, sMAD, sigma w_mean: 0.75758, 0.88782, 0.02759
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.31110, 2.33805, 2.13332
    std, sMAD, sigma w_mean: 0.53738, 0.40565, 0.02906
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.36384, 2.36108, 2.13994
	  std, sMAD, sigma w_mean: 0.51158, 0.42534, 0.02921


In [10]:
flat_alpha_r_16 = res.flat_alpha_r_16.values.copy()
flat_alpha_r_50 = res.flat_alpha_r_50.values.copy()
flat_alpha_r_84 = res.flat_alpha_r_84.values.copy()

sigma_i = (flat_alpha_r_84-flat_alpha_r_16)/2
w_i = (sigma_i)**(-2)

In [11]:
print('The following are summary statistics for alpha_r:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_lowz]), np.median(flat_alpha_r_50[normal_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_r_50[normal_lowz] - np.median(flat_alpha_r_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_lowz]), np.median(flat_alpha_r_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_lowz] - np.median(flat_alpha_r_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_host_lowz]), np.median(flat_alpha_r_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_host_lowz] - np.median(flat_alpha_r_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_r:
mean, median, weighted mean: 1.99435, 2.01032, 1.95093
std, sMAD, sigma w_mean: 0.66800, 0.60663, 0.02783
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.11799, 2.13313, 2.00928
    std, sMAD, sigma w_mean: 0.51820, 0.43057, 0.02911
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.16755, 2.24131, 2.01793
	  std, sMAD, sigma w_mean: 0.49560, 0.40529, 0.02931


In [12]:
flat_delta_16 = res.flat_delta_16.values.copy()
flat_delta_50 = res.flat_delta_50.values.copy()
flat_delta_84 = res.flat_delta_84.values.copy()

sigma_i = (flat_delta_84-flat_delta_16)/2
w_i = (sigma_i)**(-2)

In [13]:
print('The following are summary statistics for delta:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_delta_50[normal_lowz]), np.median(flat_delta_50[normal_lowz]),
                                                                   np.average(flat_delta_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_delta_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_delta_50[normal_lowz] - np.median(flat_delta_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_delta_50[normal_good_lowz]), np.median(flat_delta_50[normal_good_lowz]),
                                                                   np.average(flat_delta_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_delta_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_delta_50[normal_good_lowz] - np.median(flat_delta_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_delta_50[normal_good_host_lowz]), np.median(flat_delta_50[normal_good_host_lowz]),
                                                                   np.average(flat_delta_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_delta_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_delta_50[normal_good_host_lowz] - np.median(flat_delta_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for delta:
mean, median, weighted mean: -0.13124, -0.18258, -0.20982
std, sMAD, sigma w_mean: 0.86177, 0.16354, 0.01491
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: -0.19327, -0.18258, -0.18466
    std, sMAD, sigma w_mean: 0.08323, 0.08850, 0.01578
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: -0.19638, -0.18737, -0.18485
	  std, sMAD, sigma w_mean: 0.08519, 0.08902, 0.01582


### Repeat analysis for the t^2 SNe

In [14]:
t_rise_16 = res_tsquared.t_rise_16.values.copy()
t_rise_50 = res_tsquared.t_rise_50.values.copy()
t_rise_84 = res_tsquared.t_rise_84.values.copy()

sigma_i = (t_rise_84-t_rise_16)/2
w_i = (sigma_i)**(-2)

normal = np.where((salt_df['Ia subtype'] != 'Ia-CSM') & 
                  (salt_df['Ia subtype'] != 'SC') & 
                  (salt_df['Ia subtype'] != 'SC*') & 
                  (salt_df['Ia subtype'] != '86G-like') & 
                  (salt_df['Ia subtype'] != '02cx-like'))
good = np.where(res_tsquared.final_selection.values == 1)
has_host_z = np.where(salt_df.z_source != 'Msl')

normal_good = np.intersect1d(normal, good)
normal_good_host = np.intersect1d(normal_good, has_host_z)

normal_lowz = np.intersect1d(low_z, normal)
print(len(normal_lowz))
normal_good_lowz = np.intersect1d(low_z, normal_good)
print(len(normal_good_lowz))
normal_good_host_lowz = np.intersect1d(low_z, normal_good_host)
print(len(normal_good_host_lowz))

28
27
24


In [15]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_lowz], t_rise_50[normal_lowz], 
                            salt_df.x1_salt2_unc.values[normal_lowz], sigma_i[normal_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.5, intercept_guess=20)
print('the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_lowz])
print('the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))
salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_lowz], t_rise_50[normal_good_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_lowz], sigma_i[normal_good_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.5, intercept_guess=20)
print('    the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('    the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_lowz])
print('    the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))
print(' ')
print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

salt_uninformative = uncertain_x.UncertainAbscissa('salt')
salt_uninformative.add_data(salt_df.x1_salt2.values[normal_good_host_lowz], t_rise_50[normal_good_host_lowz], 
                            salt_df.x1_salt2_unc.values[normal_good_host_lowz], sigma_i[normal_good_host_lowz])
salt_uninformative.mle_slope_intercept(slope_guess=1.5, intercept_guess=20)
print('\t  the slope is: {:.4f} +/- {:.4f}'.format(salt_uninformative.m, salt_uninformative.m_unc))
print('\t  the intercept is: {:.4f} +/- {:.4f}'.format(salt_uninformative.b, salt_uninformative.b_unc))

residuals = t_rise_50[normal_good_host_lowz] - np.polyval([salt_uninformative.m, salt_uninformative.b], 
                                                salt_df.x1_salt2.values[normal_good_host_lowz])
print('\t  the residual scatter is: {:.4f} d'.format(np.std(residuals, ddof=1)))

The following are summary statistics for rise time:
mean, median, weighted mean: 21.02554, 21.06754, 21.67834
std, sMAD, sigma w_mean: 1.53019, 2.00500, 0.02098
the slope is: 1.6133 +/- 0.0056
the intercept is: 21.4193 +/- 0.0661
the residual scatter is: 0.7537 d
 
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 21.09506, 21.09685, 21.67924
    std, sMAD, sigma w_mean: 1.51362, 1.99175, 0.02099
    the slope is: 1.6139 +/- 0.0067
    the intercept is: 21.4190 +/- 0.1315
    the residual scatter is: 0.7673 d
 
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 21.00512, 21.06754, 21.67414
	  std, sMAD, sigma w_mean: 1.55564, 2.00500, 0.02118
	  the slope is: 1.6148 +/- 0.0297
	  the intercept is: 21.4179 +/- 0.0316
	  the residual scatter is: 0.8103 d


### Repeat analysis for the mixed sample of SNe

In [16]:
dic_df = pd.read_csv('dic_results.csv')

In [17]:
no_t2 = np.where(((dic_df.dic_evidence == 'decisive') | (dic_df.dic_evidence == 'very strong')) &  
                 (res.final_selection == 1))[0]
print(len(no_t2))

13


In [18]:
t_rise_16[no_t2] = res.t_rise_16.values.copy()[no_t2]
t_rise_50[no_t2] = res.t_rise_50.values.copy()[no_t2]
t_rise_84[no_t2] = res.t_rise_84.values.copy()[no_t2]

sigma_i = (t_rise_84-t_rise_16)/2
w_i = (sigma_i)**(-2)

In [19]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for rise time:
mean, median, weighted mean: 20.23374, 20.03293, 19.86819
std, sMAD, sigma w_mean: 1.45862, 1.46410, 0.03351
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 20.27393, 20.03496, 19.86884
    std, sMAD, sigma w_mean: 1.47052, 1.47377, 0.03352
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 20.08135, 19.92943, 19.76657
	  std, sMAD, sigma w_mean: 1.41504, 1.44402, 0.03434


### Repeat analysis for just alpha != 2 (DIC selected subsection)

In [20]:
normal_lowz = np.intersect1d(normal_lowz, no_t2)
normal_good_lowz = np.intersect1d(normal_good_lowz, no_t2)
normal_good_host_lowz = np.intersect1d(normal_good_host_lowz, no_t2)

print(len(normal_good_lowz))

9


In [21]:
print('The following are summary statistics for rise time:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_lowz]), np.median(t_rise_50[normal_lowz]),
                                                                   np.average(t_rise_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(t_rise_50[normal_lowz] - np.median(t_rise_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_lowz]), np.median(t_rise_50[normal_good_lowz]),
                                                                   np.average(t_rise_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_lowz] - np.median(t_rise_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(t_rise_50[normal_good_host_lowz]), np.median(t_rise_50[normal_good_host_lowz]),
                                                                   np.average(t_rise_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(t_rise_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(t_rise_50[normal_good_host_lowz] - np.median(t_rise_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for rise time:
mean, median, weighted mean: 19.50173, 19.82794, 19.20143
std, sMAD, sigma w_mean: 1.37550, 0.81695, 0.05686
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 19.50173, 19.82794, 19.20143
    std, sMAD, sigma w_mean: 1.37550, 0.81695, 0.05686
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 19.50173, 19.82794, 19.20143
	  std, sMAD, sigma w_mean: 1.37550, 0.81695, 0.05686


In [22]:
sigma_i = (flat_alpha_g_84-flat_alpha_g_16)/2
w_i = (sigma_i)**(-2)

print('The following are summary statistics for alpha_g:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_lowz]), np.median(flat_alpha_g_50[normal_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_g_50[normal_lowz] - np.median(flat_alpha_g_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_lowz]), np.median(flat_alpha_g_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_lowz] - np.median(flat_alpha_g_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_g_50[normal_good_host_lowz]), np.median(flat_alpha_g_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_g_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_g_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_g_50[normal_good_host_lowz] - np.median(flat_alpha_g_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_g:
mean, median, weighted mean: 2.54483, 2.50204, 2.14300
std, sMAD, sigma w_mean: 0.55384, 0.36168, 0.03123
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.54483, 2.50204, 2.14300
    std, sMAD, sigma w_mean: 0.55384, 0.36168, 0.03123
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.54483, 2.50204, 2.14300
	  std, sMAD, sigma w_mean: 0.55384, 0.36168, 0.03123


In [23]:
sigma_i = (flat_alpha_r_84-flat_alpha_r_16)/2
w_i = (sigma_i)**(-2)

print('The following are summary statistics for alpha_r:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_lowz]), np.median(flat_alpha_r_50[normal_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_alpha_r_50[normal_lowz] - np.median(flat_alpha_r_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_lowz]), np.median(flat_alpha_r_50[normal_good_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_lowz] - np.median(flat_alpha_r_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_alpha_r_50[normal_good_host_lowz]), np.median(flat_alpha_r_50[normal_good_host_lowz]),
                                                                   np.average(flat_alpha_r_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_alpha_r_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_alpha_r_50[normal_good_host_lowz] - np.median(flat_alpha_r_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for alpha_r:
mean, median, weighted mean: 2.34931, 2.42209, 2.01934
std, sMAD, sigma w_mean: 0.52445, 0.26803, 0.03148
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: 2.34931, 2.42209, 2.01934
    std, sMAD, sigma w_mean: 0.52445, 0.26803, 0.03148
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: 2.34931, 2.42209, 2.01934
	  std, sMAD, sigma w_mean: 0.52445, 0.26803, 0.03148


In [24]:
flat_delta_16 = res.flat_delta_16.values.copy()
flat_delta_50 = res.flat_delta_50.values.copy()
flat_delta_84 = res.flat_delta_84.values.copy()

sigma_i = (flat_delta_84-flat_delta_16)/2
w_i = (sigma_i)**(-2)

In [25]:
sigma_i = (flat_delta_84-flat_delta_16)/2
w_i = (sigma_i)**(-2)

print('The following are summary statistics for delta:')
print('mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_delta_50[normal_lowz]), np.median(flat_delta_50[normal_lowz]),
                                                                   np.average(flat_delta_50[normal_lowz], 
                                                                              weights=w_i[normal_lowz])))
print('std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_delta_50[normal_lowz], ddof=1), 
                                                       1.4826*np.median(np.abs(flat_delta_50[normal_lowz] - np.median(flat_delta_50[normal_lowz]))),
                                                       1/np.sum(w_i[normal_lowz])*np.sqrt(np.sum((w_i[normal_lowz]*sigma_i[normal_lowz])**2))))

print('  If you restrict to only "good" SNe however:')
print('    mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_delta_50[normal_good_lowz]), np.median(flat_delta_50[normal_good_lowz]),
                                                                   np.average(flat_delta_50[normal_good_lowz], 
                                                                              weights=w_i[normal_good_lowz])))
print('    std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_delta_50[normal_good_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_delta_50[normal_good_lowz] - np.median(flat_delta_50[normal_good_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_lowz])*np.sqrt(np.sum((w_i[normal_good_lowz]*sigma_i[normal_good_lowz])**2))))

print('\tFurther restrict to host-z SNe however:')
print('\t  mean, median, weighted mean: {:.5f}, {:.5f}, {:.5f}'.format(np.mean(flat_delta_50[normal_good_host_lowz]), np.median(flat_delta_50[normal_good_host_lowz]),
                                                                   np.average(flat_delta_50[normal_good_host_lowz], 
                                                                              weights=w_i[normal_good_host_lowz])))
print('\t  std, sMAD, sigma w_mean: {:.5f}, {:.5f}, {:.5f}'.format(np.std(flat_delta_50[normal_good_host_lowz], ddof=1), 
                                                                   1.4826*np.median(np.abs(flat_delta_50[normal_good_host_lowz] - np.median(flat_delta_50[normal_good_host_lowz]))), 
                                                                   1/np.sum(w_i[normal_good_host_lowz])*np.sqrt(np.sum((w_i[normal_good_host_lowz]*sigma_i[normal_good_host_lowz])**2))))

The following are summary statistics for delta:
mean, median, weighted mean: -0.19834, -0.21839, -0.18702
std, sMAD, sigma w_mean: 0.07966, 0.06019, 0.01701
  If you restrict to only "good" SNe however:
    mean, median, weighted mean: -0.19834, -0.21839, -0.18702
    std, sMAD, sigma w_mean: 0.07966, 0.06019, 0.01701
	Further restrict to host-z SNe however:
	  mean, median, weighted mean: -0.19834, -0.21839, -0.18702
	  std, sMAD, sigma w_mean: 0.07966, 0.06019, 0.01701
